In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path
import itertools

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_base import ProjectedGradientDescentBase, ProjectedGradientDescentBaseOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement, get_z_measurement
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)
from quara.objects.operators import tensor_product

from quara.data_analysis.simulation import SimulationSetting

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys_1 = ElementalSystem(0, get_normalized_pauli_basis())
c_sys_1 = CompositeSystem([e_sys_1])

e_sys_2 = ElementalSystem(1, get_normalized_pauli_basis())
c_sys_2 = CompositeSystem([e_sys_2])

# |+><+|
# state_x0 = get_x0_1q(c_sys)
# |+i><+i|
# state_y0 = get_y0_1q(c_sys)
# |0><0|
# state_z0 = get_z0_1q(c_sys)
# |1><1|
# state_z1 = get_z1_1q(c_sys)

tester_objects = []

# |+><+| |+i><+i| |0><0| |1><1|
func_list = [get_x0_1q, get_y0_1q, get_z0_1q, get_z1_1q]

for i, funcs in enumerate(itertools.product(func_list, func_list)):
    state1 = funcs[0](c_sys_1)
    state2 = funcs[1](c_sys_2)

    state_2qubit = tensor_product(state1, state2)

    tester_objects.append(state_2qubit)

In [5]:
true_object = tensor_product( get_z_measurement(c_sys_1),  get_z_measurement(c_sys_2))

In [6]:
measurement_n = len(true_object.vecs)
seed = 777
povmt = StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed)

m=4


In [7]:
num_data = [100, 1000]
n_rep = 10
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator()
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]

    start = time.time()

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        iteration=n_rep,
    )

    # stock settings of this simulation
    simulation_setting = SimulationSetting(
        name=name,
        estimator=estimator
    )
    print(simulation_setting)
    simulation_settings.append(simulation_setting)

    estimation_results_list.append(estimation_results)

    elapsed_time = time.time() - start
    print("elapsed_time: {0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

m=4


  0%|          | 0/10 [00:00<?, ?it/s]

m=4
m=4
m=4


 10%|█         | 1/10 [00:00<00:01,  6.48it/s]

Name: LinearEstimator(True)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.028486414750417074[min]



...

Name: LinearEstimator(False)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.02265985409418742[min]



  0%|          | 0/10 [00:00<?, ?it/s]

Name: ProjectedLinearEstimator(True)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 0.8997965852419535[min]



100%|██████████| 10/10 [00:50<00:00,  5.06s/it]

Name: ProjectedLinearEstimator(False)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 0.8439060688018799[min]



In [8]:
report.export_report("povmt_2qubit.pdf",
    estimation_results_list,
    simulation_settings,
    true_object,
    tester_objects,
    seed=seed,
    computation_time=sum(elapsed_times)
)

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...


100%|██████████| 10/10 [00:00<00:00, 20961.04it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  LinearEstimator(True)=[0.2533399999999997, 0.029085400000000074]
  LinearEstimator(False)=[0.27093999999999996, 0.030259400000000092]
  ProjectedLinearEstimator(True)=[0.15539404582940464, 0.018833548125652022]
  ProjectedLinearEstimator(False)=[0.15464602493725754, 0.018579593481549174]
{'make_graphs_func': <function make_mses_graphs_estimator at 0x10bfa4940>, 'mse_type': 'estimator', 'estimation_results_list': [[<quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11a90>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce119d0>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11c10>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11ca0>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11c40>, <q

100%|██████████| 10/10 [00:00<00:00, 6869.15it/s]

{'make_graphs_func': <function make_mses_graphs_para at 0x10bfa49d0>, 'mse_type': 'para', 'estimation_results_list': [[<quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11a90>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce119d0>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11c10>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11ca0>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11c40>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11b20>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11a00>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x10ce11b50>, <quara.protocol.qtomography.standard.linear_estimator.LinearEstimationRes


  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 62.21it/s]

0it [00:00, ?it/s]
4it [00:00, 15.92it/s]
 50%|█████     | 2/4 [00:00<00:00,  5.97it/s]
4it [00:00, 47.29it/s]
 75%|███████▌  | 3/4 [00:00<00:00,  6.77it/s]
4it [00:00, 46.65it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 50.61it/s]

4it [00:00, 44.00it/s]
 50%|█████     | 2/4 [00:00<00:00, 10.70it/s]
4it [00:00, 49.25it/s]

4it [00:00, 58.50it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 43.83it/s]
 25%|██▌       | 1/4 [00:00<00:00,  9.69it/s]
0it [00:00, ?it/s]
4it [00:00, 28.22it/s]
 50%|█████     | 2/4 [00:00<00:00,  5.91it/s]
4it [00:00, 47.39it/s]
 75%|███████▌  | 3/4 [00:00<00:00,  6.68it/s]
4it [00:00, 50.61it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 46.05it/s]

4it [00:00, 53.87it/s]
 50%|█████     | 2/4 [00:00<00:00, 11.36it/s]
4it [00:00, 43.41it/s]
 75%|███████▌  | 3/4 [00:00<00:00, 10.87it/s]
4it [00:00, 52.08it/s]
  0%|          | 0/4 [00:00<?, ?it/s]
4it [00:00, 50.03it/s]

4it [00

Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (povmt_2qubit_20210115_修正後.pdf)
